In [6]:
import torch as th
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import sys
if "../FFF" not in sys.path:
    sys.path.insert(0, "../FFF")

%load_ext autoreload
%autoreload 2
from fff import FFF

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1024,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [22]:
class Net_regular(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Net_FFF(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = FFF(16 * 5 * 5, 120)
        self.fc2 = FFF(120, 84)
        self.fc3 = FFF(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [23]:

def evaluate(net: nn.Module, data_loader: torch.utils.data.DataLoader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
    return 100 * correct / total

def train(net: nn.Module,
          trainloader: torch.utils.data.DataLoader,
          testloader: torch.utils.data.DataLoader,
          epochs: int):

    optimizer = optim.AdamW(net.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch} loss: {running_loss / len(trainloader)}")
        print(f"Train accuracy: {evaluate(net, trainloader)}")
        print(f"Test accuracy: {evaluate(net, testloader)}")

In [24]:
net_regular = Net_regular()
net_FFF = Net_FFF()

print('eval net_regular:', evaluate(net_regular, testloader))
print('eval net_FFF:', evaluate(net_FFF, testloader))

eval net_regular: tensor(9.7900)
eval net_FFF: tensor(9.8100)


In [25]:
train(net_regular, trainloader, testloader, 3)

Epoch 0 loss: 1.7309615273609795
Train accuracy: 44.762001037597656
Test accuracy: 44.91999816894531
Epoch 1 loss: 1.459026979363483
Train accuracy: 48.7859992980957
Test accuracy: 47.7599983215332
Epoch 2 loss: 1.3532817153369678
Train accuracy: 53.36199951171875
Test accuracy: 52.0099983215332


In [26]:
train(net_FFF, trainloader, testloader, 3)

Epoch 0 loss: 1.9808714978225397
Train accuracy: 33.970001220703125
Test accuracy: 34.43000030517578
Epoch 1 loss: 1.6775692061085226
Train accuracy: 42.071998596191406
Test accuracy: 41.86000061035156
Epoch 2 loss: 1.5615577337991855
Train accuracy: 44.948001861572266
Test accuracy: 43.90999984741211


In [27]:
%%timeit
evaluate(net_regular, testloader)

5.94 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
evaluate(net_FFF, testloader)

6.3 s ± 223 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
